<a href="https://colab.research.google.com/github/alexsvgithub/Text-Summarization/blob/main/PBL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
df = input("Enter the text to summarize")
print(df)
len(df)

Enter the text to summarizeChristianity is an Abrahamic monotheistic religion based on the life and teachings of Jesus of Nazareth. It is the world's largest religion, with about 2.4 billion followers.[1] Its adherents, known as Christians, make up a majority of the population in 157 countries and territories,[2] and believe that Jesus is the Christ, whose coming as the Messiah was prophesied in the Hebrew Bible, called the Old Testament in Christianity, and chronicled in the New Testament.[3]  Christianity remains culturally diverse in its Western and Eastern branches, as well as in its doctrines concerning justification and the nature of salvation, ecclesiology, ordination, and Christology. Their creeds generally hold in common Jesus as the Son of God—the Logos incarnated—who ministered, suffered, and died on a cross, but rose from the dead for the salvation of mankind; and referred to as the gospel, meaning the "good news". Describing Jesus' life and teachings are the four canonical

In [8]:
from nltk.tokenize import sent_tokenize
sentences = []

sentences.append(sent_tokenize(df))

#sentences = [y for x in sentences for y in x] # flattern list(Seperate on the basis of fullstop (.))

print(sentences)

[['Christianity is an Abrahamic monotheistic religion based on the life and teachings of Jesus of Nazareth.', "It is the world's largest religion, with about 2.4 billion followers.", '[1] Its adherents, known as Christians, make up a majority of the population in 157 countries and territories,[2] and believe that Jesus is the Christ, whose coming as the Messiah was prophesied in the Hebrew Bible, called the Old Testament in Christianity, and chronicled in the New Testament.', '[3]  Christianity remains culturally diverse in its Western and Eastern branches, as well as in its doctrines concerning justification and the nature of salvation, ecclesiology, ordination, and Christology.', 'Their creeds generally hold in common Jesus as the Son of God—the Logos incarnated—who ministered, suffered, and died on a cross, but rose from the dead for the salvation of mankind; and referred to as the gospel, meaning the "good news".', "Describing Jesus' life and teachings are the four canonical gospel

In [9]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [10]:
len(word_embeddings)

400000

In [11]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [df.lower() for s in clean_sentences]

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [14]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [15]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [16]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [17]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [18]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [21]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [34]:
ranked_sentences = sorted(((scores[i],df) for i,df in enumerate(sentences)), reverse=True)
#print(ranked_sentences[0][1])

In [66]:
final = []
# Extract top 10 sentences as the summary
for i in range (1,10):
  final.append(ranked_sentences[0][1][i])
print(final)
len(df)
len(final)

["It is the world's largest religion, with about 2.4 billion followers.", '[1] Its adherents, known as Christians, make up a majority of the population in 157 countries and territories,[2] and believe that Jesus is the Christ, whose coming as the Messiah was prophesied in the Hebrew Bible, called the Old Testament in Christianity, and chronicled in the New Testament.', '[3]  Christianity remains culturally diverse in its Western and Eastern branches, as well as in its doctrines concerning justification and the nature of salvation, ecclesiology, ordination, and Christology.', 'Their creeds generally hold in common Jesus as the Son of God—the Logos incarnated—who ministered, suffered, and died on a cross, but rose from the dead for the salvation of mankind; and referred to as the gospel, meaning the "good news".', "Describing Jesus' life and teachings are the four canonical gospels of Matthew, Mark, Luke and John, with the Old Testament as the gospel's respected background.", 'Christiani

9